In [4]:
import zipfile
import os

zip_path = "/content/Training.zip"
extract_dir = "/content/Training"  # Changed from Training.zip to Training

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted to: {extract_dir}")

Extracted to: /content/Training


In [8]:
from pathlib import Path

TRAIN_DIR = Path("/content/Training/Training")

print("Subfolders found:")
for d in TRAIN_DIR.iterdir():
    if d.is_dir():
        print("  ", d.name)

Subfolders found:
   pituitary_tumor
   meningioma_tumor
   glioma_tumor
   no_tumor


In [9]:
print("Image counts in each class:")
for d in TRAIN_DIR.iterdir():
    if d.is_dir():
        count = len(list(d.glob("*")))
        print(f"{d.name}: {count}")

Image counts in each class:
pituitary_tumor: 827
meningioma_tumor: 822
glioma_tumor: 826
no_tumor: 395


In [10]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from pathlib import Path

PROCESSED_DIR = Path("/content/processed_training")
PROCESSED_DIR.mkdir(exist_ok=True)

IMG_SIZE = (224, 224)
rows = []

supported_ext = (".jpg", ".png", ".jpeg", ".bmp", ".tif", ".tiff")

for class_dir in sorted(TRAIN_DIR.iterdir()):
    if not class_dir.is_dir():
        continue

    label = class_dir.name
    print("Processing:", label)

    out_dir = PROCESSED_DIR / label
    out_dir.mkdir(exist_ok=True)

    images = [p for p in class_dir.iterdir() if p.suffix.lower() in supported_ext]

    for img_path in tqdm(images, desc=f"{label}", unit="img"):
        # Read image (imdecode avoids Unicode path issues)
        img = cv2.imdecode(np.fromfile(str(img_path), dtype=np.uint8), cv2.IMREAD_COLOR)
        if img is None:
            continue

        orig_h, orig_w = img.shape[:2]

        # Resize
        resized = cv2.resize(img, IMG_SIZE)

        # Normalize
        norm = resized.astype("float32") / 255.0

        # Save processed image
        out_path = out_dir / f"{img_path.stem}.png"
        cv2.imwrite(str(out_path), (norm * 255).astype("uint8"))

        # Metadata entry
        rows.append({
            "original_path": str(img_path),
            "processed_path": str(out_path),
            "label": label,
            "orig_width": orig_w,
            "orig_height": orig_h,
            "proc_width": IMG_SIZE[0],
            "proc_height": IMG_SIZE[1],
            "mean_pixel": float(norm.mean()),
            "std_pixel": float(norm.std()),
            "ingest_timestamp": datetime.utcnow().isoformat()
        })

meta_df = pd.DataFrame(rows)
meta_df.to_csv("/content/metadata.csv", index=False)

print("ETL completed.")
meta_df.head()

Processing: glioma_tumor


glioma_tumor:   0%|          | 0/826 [00:00<?, ?img/s]/tmp/ipython-input-640561310.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ingest_timestamp": datetime.utcnow().isoformat()
glioma_tumor: 100%|██████████| 826/826 [00:06<00:00, 127.72img/s]


Processing: meningioma_tumor


meningioma_tumor: 100%|██████████| 822/822 [00:04<00:00, 176.07img/s]


Processing: no_tumor


no_tumor: 100%|██████████| 395/395 [00:02<00:00, 175.69img/s]


Processing: pituitary_tumor


pituitary_tumor: 100%|██████████| 827/827 [00:06<00:00, 129.92img/s]


ETL completed.


,original_path,processed_path,label,orig_width,orig_height,proc_width,proc_height,mean_pixel,std_pixel,ingest_timestamp
0,/content/Training/Training/glioma_tumor/gg (33...,/content/processed_training/glioma_tumor/gg (3...,glioma_tumor,512,512,224,224,0.121688,0.124105,2025-11-29T21:51:10.844692
1,/content/Training/Training/glioma_tumor/gg (64...,/content/processed_training/glioma_tumor/gg (6...,glioma_tumor,512,512,224,224,0.091847,0.123684,2025-11-29T21:51:10.853064
2,/content/Training/Training/glioma_tumor/gg (23...,/content/processed_training/glioma_tumor/gg (2...,glioma_tumor,512,512,224,224,0.146762,0.149962,2025-11-29T21:51:10.861241
3,/content/Training/Training/glioma_tumor/gg (63...,/content/processed_training/glioma_tumor/gg (6...,glioma_tumor,512,512,224,224,0.097496,0.142852,2025-11-29T21:51:10.877090
4,/content/Training/Training/glioma_tumor/gg (18...,/content/processed_training/glioma_tumor/gg (1...,glioma_tumor,512,512,224,224,0.186449,0.164861,2025-11-29T21:51:10.886061


In [12]:
import sqlite3

db_path = "/content/brain_etl.db"
conn = sqlite3.connect(db_path)

# Load metadata into DB
meta_df.to_sql("mri_scans", conn, if_exists="replace", index=False)

# Table for model predictions (populate later)
conn.execute("""
CREATE TABLE IF NOT EXISTS tumor_classification (
    classification_id INTEGER PRIMARY KEY AUTOINCREMENT,
    processed_path TEXT,
    predicted_label TEXT,
    confidence FLOAT,
    model_name TEXT,
    classified_on TEXT
);
""")

conn.commit()

print("Database created at:", db_path)

Database created at: /content/brain_etl.db


In [20]:
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import random

conn = sqlite3.connect("/content/brain_etl.db")

# Load the table
df = pd.read_sql_query("SELECT * FROM mri_scans;", conn)

# Add synthetic fields
np.random.seed(42)

df['patient_id'] = np.random.randint(1000, 9999, size=len(df))
df['age'] = np.random.randint(20, 80, size=len(df))
df['gender'] = np.random.choice(['M', 'F'], size=len(df))
df['hospital_unit'] = np.random.choice(
    ['Neuro', 'Radiology', 'Oncology'],
    size=len(df),
    p=[0.5, 0.3, 0.2]
)

# Random scan dates within last 2 years
start_date = datetime.now() - timedelta(days=730)
df['scan_date'] = [
    (start_date + timedelta(days=random.randint(0, 730))).strftime("%Y-%m-%d")
    for _ in range(len(df))
]

# Save back into DB
df.to_sql("mri_scans", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print("Synthetic demographic fields successfully added!")
df.head()

Synthetic demographic fields successfully added!


,original_path,processed_path,label,orig_width,orig_height,proc_width,proc_height,mean_pixel,std_pixel,ingest_timestamp,patient_id,age,gender,hospital_unit,scan_date
0,/content/Training/Training/glioma_tumor/gg (33...,/content/processed_training/glioma_tumor/gg (3...,glioma_tumor,512,512,224,224,0.121688,0.124105,2025-11-29T21:51:10.844692,8270,24,M,Neuro,2024-04-15
1,/content/Training/Training/glioma_tumor/gg (64...,/content/processed_training/glioma_tumor/gg (6...,glioma_tumor,512,512,224,224,0.091847,0.123684,2025-11-29T21:51:10.853064,1860,63,F,Neuro,2024-05-18
2,/content/Training/Training/glioma_tumor/gg (23...,/content/processed_training/glioma_tumor/gg (2...,glioma_tumor,512,512,224,224,0.146762,0.149962,2025-11-29T21:51:10.861241,6390,76,F,Neuro,2024-01-27
3,/content/Training/Training/glioma_tumor/gg (63...,/content/processed_training/glioma_tumor/gg (6...,glioma_tumor,512,512,224,224,0.097496,0.142852,2025-11-29T21:51:10.877090,6191,48,M,Oncology,2024-03-31
4,/content/Training/Training/glioma_tumor/gg (18...,/content/processed_training/glioma_tumor/gg (1...,glioma_tumor,512,512,224,224,0.186449,0.164861,2025-11-29T21:51:10.886061,6734,47,M,Radiology,2025-08-17


In [24]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("/content/brain_etl.db")

In [26]:
pd.read_sql_query("""
SELECT label AS tumor_type,
       COUNT(*) AS total_cases
FROM mri_scans
GROUP BY label
ORDER BY total_cases DESC;
""", conn)

,tumor_type,total_cases
0,pituitary_tumor,827
1,glioma_tumor,826
2,meningioma_tumor,822
3,no_tumor,395


In [28]:
pd.read_sql_query("""
SELECT
    CASE
        WHEN age BETWEEN 20 AND 29 THEN '20s'
        WHEN age BETWEEN 30 AND 39 THEN '30s'
        WHEN age BETWEEN 40 AND 49 THEN '40s'
        WHEN age BETWEEN 50 AND 59 THEN '50s'
        WHEN age BETWEEN 60 AND 69 THEN '60s'
        ELSE '70s+'
    END AS age_group,
    label,
    AVG(orig_width * orig_height) AS avg_area,
    COUNT(*) AS cases
FROM mri_scans
GROUP BY age_group, label
ORDER BY age_group;
""", conn)

,age_group,label,avg_area,cases
0,20s,glioma_tumor,262144.000000,134
1,20s,meningioma_tumor,243278.270677,133
2,20s,no_tumor,144884.979592,49
3,20s,pituitary_tumor,256257.526718,131
4,30s,glioma_tumor,262144.000000,139
5,30s,meningioma_tumor,244152.863636,132
6,30s,no_tumor,148098.519481,77
7,30s,pituitary_tumor,265086.013245,151
8,40s,glioma_tumor,262144.000000,142
9,40s,meningioma_tumor,229341.965517,145


In [30]:
pd.read_sql_query("""
SELECT hospital_unit,
       label AS tumor_type,
       COUNT(*) AS cases
FROM mri_scans
GROUP BY hospital_unit, label
ORDER BY hospital_unit, cases DESC;
""", conn)

,hospital_unit,tumor_type,cases
0,Neuro,pituitary_tumor,441
1,Neuro,meningioma_tumor,421
2,Neuro,glioma_tumor,417
3,Neuro,no_tumor,211
4,Oncology,meningioma_tumor,174
5,Oncology,glioma_tumor,166
6,Oncology,pituitary_tumor,145
7,Oncology,no_tumor,71
8,Radiology,glioma_tumor,243
9,Radiology,pituitary_tumor,241


In [31]:
pd.read_sql_query("""
SELECT gender, label AS tumor_type, COUNT(*) AS total
FROM mri_scans
GROUP BY gender, tumor_type
ORDER BY gender;
""", conn)

,gender,tumor_type,total
0,F,glioma_tumor,392
1,F,meningioma_tumor,408
2,F,no_tumor,223
3,F,pituitary_tumor,404
4,M,glioma_tumor,434
5,M,meningioma_tumor,414
6,M,no_tumor,172
7,M,pituitary_tumor,423


In [32]:
pd.read_sql_query("""
SELECT processed_path, age, gender, hospital_unit
FROM mri_scans
WHERE label = 'glioma_tumor'
  AND age > 50
LIMIT 100;
""", conn)

,processed_path,age,gender,hospital_unit
0,/content/processed_training/glioma_tumor/gg (6...,63,F,Neuro
1,/content/processed_training/glioma_tumor/gg (2...,76,F,Neuro
2,/content/processed_training/glioma_tumor/gg (3...,76,F,Neuro
3,/content/processed_training/glioma_tumor/gg (3...,62,F,Neuro
4,/content/processed_training/glioma_tumor/gg (5...,53,M,Radiology
...,...,...,...,...
95,/content/processed_training/glioma_tumor/gg (7...,51,M,Oncology
96,/content/processed_training/glioma_tumor/gg (2...,72,F,Radiology
97,/content/processed_training/glioma_tumor/gg (3...,52,F,Neuro
98,/content/processed_training/glioma_tumor/gg (5...,74,M,Radiology


In [33]:
pd.read_sql_query("""
SELECT
    STRFTIME('%Y-%W', scan_date) AS week,
    label AS tumor_type,
    COUNT(*) AS cases
FROM mri_scans
GROUP BY week, tumor_type
ORDER BY week;
""", conn)

,week,tumor_type,cases
0,2023-48,glioma_tumor,4
1,2023-48,meningioma_tumor,6
2,2023-48,no_tumor,3
3,2023-48,pituitary_tumor,8
4,2023-49,glioma_tumor,6
...,...,...,...
415,2025-46,pituitary_tumor,9
416,2025-47,glioma_tumor,6
417,2025-47,meningioma_tumor,6
418,2025-47,no_tumor,5
